In [27]:
import pandas as pd
from dotenv import load_dotenv
import os
import numpy as np
from scipy.optimize import curve_fit

from sqlalchemy import create_engine
from sqlalchemy import text

from matplotlib import pyplot as plt
load_dotenv()

user = os.environ.get("DB_USERNAME")
password = os.environ.get("DB_PASSWORD")
hostname = os.environ.get("DB_HOST")
database_name =  os.environ.get("DB_NAME")
port = os.environ.get("DB_PORT")
cluster = os.environ.get("DB_CLUSTER")
youtube_api_key = os.environ.get("YOUTUBE_API_KEY")

engine = create_engine(f'cockroachdb://{user}:{password}@{hostname}:{port}/{database_name}?sslmode=require&options=--cluster={cluster}')


In [30]:

df = None
with engine.connect() as conn:
    df = pd.read_sql("SELECT * FROM LIVE_STREAMS", conn)

df = df.set_index(df["log_time"].dt.tz_localize("UTC").dt.tz_convert("US/Pacific"))
    
df_aggreagtes = pd.DataFrame()
df_aggreagtes["streamed_hours"] = df.resample("1D")["is_live"].apply(lambda x: (x == True).sum() / 6)

df_aggreagtes["utc_datetime"] = df_aggreagtes.index

df_aggreagtes["utc_datetime"] = df_aggreagtes["utc_datetime"].apply(lambda x: str(x)[:-6])


In [31]:
print(df_aggreagtes.head())

                           streamed_hours         utc_datetime
log_time                                                      
2022-03-28 00:00:00-07:00        3.333333  2022-03-28 00:00:00
2022-03-29 00:00:00-07:00        5.333333  2022-03-29 00:00:00
2022-03-30 00:00:00-07:00        6.333333  2022-03-30 00:00:00
2022-03-31 00:00:00-07:00        4.166667  2022-03-31 00:00:00
2022-04-01 00:00:00-07:00        0.000000  2022-04-01 00:00:00


In [33]:
df_aggreagtes.to_csv("total_ludwig_hours.csv")